In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
#read in wine csv
wine_df=pd.read_csv("../Resources/Winedata.csv")
wine_df.head()

In [ ]:
# read weather csv from S3 - which is super cool, thanks Marisa
weather_df=pd.read_csv("https://grp2weather.s3.amazonaws.com/combined_weather.csv")
weather_df.head()

In [ ]:
#extract year column from title field, pop title
wine_df['Year']=wine_df['title'].str.extract(r'([1-2]\d{3})')
wine_df.pop('title')
wine_df.head()

In [ ]:
#filter wine_df for regions we want to examine
regions=['California','Washington','Bordeaux','Tuscany','Oregon','Burgundy','Cantabria','Piedmont','Veneto','New York','Alsace','Sicily & Sardinia','Champagne']
wine_df_regions=wine_df[wine_df['province'].isin(regions)]
wine_df_regions

In [ ]:
# drop records with no year
wine_df_regions.dropna(subset=['Year'],inplace=True)
wine_df_regions

In [ ]:
#drop records from before 1990
wine_df_regions=wine_df_regions[wine_df_regions['Year']>'1990']
wine_df_regions

In [ ]:
#make list of columns to drop and drop them
columns_to_drop=['country','description','designation','price','region_2','region_1','winery']
wine_df_regions.drop(columns_to_drop,axis=1,inplace=True)
wine_df_regions

In [ ]:
wine_df_regions.columns

### Target
- points

### Variables
- province - categorical 
- region_1 - categorical 
- variety - categorical 
- winery - categorical 
- Year - categorical 

### Maybe Not Variables
- description
- designation
- price

### Definitely Not Variables
- id
- country
- region_2

In [ ]:
#replace wine region values with matching regions from weather data
#Piedmont needs to be Piemonte
wine_df_regions.loc[wine_df_regions['province']=='Piedmont','province']="Piemonte"
wine_df_regions.loc[wine_df_regions['province']=='Sicily & Sardinia','province']="Sicilia"
wine_df_regions.loc[wine_df_regions['province']=='Bordeaux','province']="Aquitaine"
wine_df_regions.loc[wine_df_regions['province']=='Northern Spain','province']="Cantabria"
wine_df_regions.loc[wine_df_regions['province']=='Champagne','province']="Champagne-Ardenne"
wine_df_regions

In [ ]:
wine_df_regions[wine_df_regions['province']=='Piemonte']

In [ ]:
variety_vcs=wine_df_regions['variety'].value_counts()
variety_vcs
#Bin variety to elminate <2200

In [ ]:
variety_counts=wine_df_regions.variety.value_counts()
replace_varieties=list(variety_counts[variety_counts<2200].index)
replace_varieties

In [ ]:
variety_vcs=wine_df_regions['variety'].unique()

In [ ]:
variety_vcs